<a href="https://colab.research.google.com/github/OmdenaAI/trieste-italy-long-covid/blob/main/src/tasks/task-6-sentiment_analysis-text_classification-topic_modeling/xlm_t_zeroshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inference Only (Zero-shot) XLM-T Model 

This is the code for an inference-only XLM-T (base) model on the sample of 50 tweets that have been manually labeled(Sentiment Analysis). There is no training involved. 
Labels:
* Label 0: Negative
* Label 1: Neutral
* Label 2: Positive



### Accuracy Metrics
* Weighted Average Precision: 0.73
* Macro Average Precision: 0.54
* Weighted Average F1 Score: 0.53

In [ ]:
import numpy as np
import pandas as pd
import torch

import random

In [ ]:
!pip install --upgrade pip
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 46.1 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13

In [ ]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import pipeline
from collections import defaultdict 
import urllib
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

from scipy.special import softmax
from sklearn.metrics import classification_report

In [ ]:
random.seed(42)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/trieste-italy-long-covid/labeled_LCovid_sentiment_Batch_E2.csv')
df.head()

,Tweet_id,Tweet,sent_label,class_label
0,"1,39134418176387E+018",#Covid19\nVillorba\nAnziana 91 anni muore nel ...,0,0
1,"1,3913692381962E+018",Lo capite vero che il problema non è il covid1...,0,0
2,"1,39141225080469E+018",#Newsicilia Il #longCovid riguarda i soggetti ...,0,1
3,"1,39142375468585E+018","@MMmarco0 Cazzo,un tuo tweet non votato all'al...",0,0
4,"1,39147439471159E+018","@G23Mld Mah... è la solita minimizzazione: ""è ...",0,1


In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)





In [ ]:
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
#tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
config = AutoConfig.from_pretrained(MODEL) # used for id to label name
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [ ]:
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"


tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
config = AutoConfig.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(model_path)
#tokenizer.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

In [ ]:
sample_text = df.Tweet.iloc[0]
sample_text

'#Covid19\nVillorba\nAnziana 91 anni muore nel parcheggio del centro vaccinale 1h dopo la 2ª dose del vaccino #Pfizer\nL’anziana comunque soffriva di patologie cardiovascolari e renali.\nDecesso CON vaccino\nDisposta autopsia. https://t.co/fifpwW2OwA'

In [ ]:
text = "Good night 😊"
text = preprocess(sample_text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
print(f'score is {scores}')
scores = softmax(scores)



# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    #print(f'the l value is {l}')
    #print(f'the s value is {s}')
    print(f"{i+1}) {l} {np.round(float(s), 4)}")
#print(f'the r value is {r}')

score is [ 1.5974543   0.07005377 -2.2853854 ]
1) Negative 0.808
2) Neutral 0.1754
3) Positive 0.0166


In [ ]:
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task(sample_text)


[{'label': 'Negative', 'score': 0.7890516519546509}]

In [ ]:
tweets = df['Tweet'].tolist()
len(tweets)

50

In [ ]:
tweets_proc = []
for tweet in tweets:
    tweets_proc.append(preprocess(tweet))

len(tweets_proc)

50

In [ ]:
all_preds = []
all_probs = []
all_preds_labels = []

for tweet_a in tweets_proc:
    encoded_input = tokenizer(tweet_a, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    m = np.argmax(scores)
    #print(m)
    #print(config.id2label[m])
    #print(scores[m])
    all_preds_labels.append(config.id2label[m])
    all_preds.append(m)
    all_probs.append(scores[m])

   



In [ ]:
print(all_preds)


[0, 0, 1, 0, 0, 2, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]


In [ ]:
print(all_probs)

[0.8079557, 0.8904591, 0.8580968, 0.9550231, 0.83148354, 0.5005461, 0.5049496, 0.5984179, 0.54688215, 0.72662854, 0.9543896, 0.9436496, 0.9035031, 0.7264677, 0.64913476, 0.9607226, 0.84084207, 0.5524689, 0.7250103, 0.8869803, 0.84798694, 0.45183197, 0.8969673, 0.9316223, 0.8494254, 0.53031707, 0.53031707, 0.9057248, 0.8577045, 0.8577045, 0.558061, 0.89553803, 0.8553888, 0.85512984, 0.7099021, 0.8343968, 0.89712644, 0.82642967, 0.8188391, 0.93353814, 0.48233813, 0.46720943, 0.70356804, 0.935385, 0.5568394, 0.5056935, 0.9081746, 0.94137496, 0.938495, 0.86971575]


In [ ]:
df['xlm-t-model-preds'] = all_preds
df

,Tweet_id,Tweet,sent_label,class_label,xlm-t-model-preds
0,"1,39134418176387E+018",#Covid19\nVillorba\nAnziana 91 anni muore nel ...,0,0,0
1,"1,3913692381962E+018",Lo capite vero che il problema non è il covid1...,0,0,0
2,"1,39141225080469E+018",#Newsicilia Il #longCovid riguarda i soggetti ...,0,1,1
3,"1,39142375468585E+018","@MMmarco0 Cazzo,un tuo tweet non votato all'al...",0,0,0
4,"1,39147439471159E+018","@G23Mld Mah... è la solita minimizzazione: ""è ...",0,1,0
5,"1,39150509820813E+018",#Ashwagandha produce ottimi risultati su stre...,0,0,2
6,"1,39152716121186E+018",@Regsprecher L'UE deve agire con decisione per...,0,0,2
7,"1,39635772657437E+018",Svizzera: « In ambito scolastico un bambino su...,0,1,1
8,"1,3963620985785E+018",@Aifa_ufficiale ci sono controindicazioni ai v...,0,0,0
9,"1,39636686239772E+018",Al rischio del contagio da #COVID19 si aggiung...,1,1,1


In [ ]:
df['sent_label'] = df['sent_label'] + 1  
df

,Tweet_id,Tweet,sent_label,class_label,xlm-t-model-preds
0,"1,39134418176387E+018",#Covid19\nVillorba\nAnziana 91 anni muore nel ...,1,0,0
1,"1,3913692381962E+018",Lo capite vero che il problema non è il covid1...,1,0,0
2,"1,39141225080469E+018",#Newsicilia Il #longCovid riguarda i soggetti ...,1,1,1
3,"1,39142375468585E+018","@MMmarco0 Cazzo,un tuo tweet non votato all'al...",1,0,0
4,"1,39147439471159E+018","@G23Mld Mah... è la solita minimizzazione: ""è ...",1,1,0
5,"1,39150509820813E+018",#Ashwagandha produce ottimi risultati su stre...,1,0,2
6,"1,39152716121186E+018",@Regsprecher L'UE deve agire con decisione per...,1,0,2
7,"1,39635772657437E+018",Svizzera: « In ambito scolastico un bambino su...,1,1,1
8,"1,3963620985785E+018",@Aifa_ufficiale ci sono controindicazioni ai v...,1,0,0
9,"1,39636686239772E+018",Al rischio del contagio da #COVID19 si aggiung...,2,1,1


In [ ]:
annot_preds = list(df['sent_label'])
print(classification_report(annot_preds, all_preds))

              precision    recall  f1-score   support

           0       0.51      1.00      0.68        18
           1       0.90      0.30      0.45        30
           2       0.20      0.50      0.29         2

    accuracy                           0.56        50
   macro avg       0.54      0.60      0.47        50
weighted avg       0.73      0.56      0.53        50



In [ ]:
df

,Tweet_id,Tweet,sent_label,class_label,xlm-t-model-preds
0,"1,39134418176387E+018",#Covid19\nVillorba\nAnziana 91 anni muore nel ...,1,0,0
1,"1,3913692381962E+018",Lo capite vero che il problema non è il covid1...,1,0,0
2,"1,39141225080469E+018",#Newsicilia Il #longCovid riguarda i soggetti ...,1,1,1
3,"1,39142375468585E+018","@MMmarco0 Cazzo,un tuo tweet non votato all'al...",1,0,0
4,"1,39147439471159E+018","@G23Mld Mah... è la solita minimizzazione: ""è ...",1,1,0
5,"1,39150509820813E+018",#Ashwagandha produce ottimi risultati su stre...,1,0,2
6,"1,39152716121186E+018",@Regsprecher L'UE deve agire con decisione per...,1,0,2
7,"1,39635772657437E+018",Svizzera: « In ambito scolastico un bambino su...,1,1,1
8,"1,3963620985785E+018",@Aifa_ufficiale ci sono controindicazioni ai v...,1,0,0
9,"1,39636686239772E+018",Al rischio del contagio da #COVID19 si aggiung...,2,1,1


In [ ]:
df.to_csv('/content/drive/MyDrive/trieste-italy-long-covid/xlm-inference-labels-final.csv', index=False)